In [1]:
%pip install transformers torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install transformers peft datasets bitsandbytes accelerate ipywidgets


Note: you may need to restart the kernel to use updated packages.


In [3]:
# 모델을 직접 로드하여 사용
from transformers import AutoTokenizer, AutoModelForCausalLM

# 토크나이저 및 모델 로드
tokenizer = AutoTokenizer.from_pretrained("kakaocorp/kanana-nano-2.1b-instruct")
model = AutoModelForCausalLM.from_pretrained("kakaocorp/kanana-nano-2.1b-instruct")

# 텍스트를 토크나이즈하여 모델 입력 형태로 변환
input_text = "Who are you?"
inputs = tokenizer(input_text, return_tensors="pt")

# 모델을 사용하여 출력 생성
outputs = model.generate(inputs["input_ids"], max_length=50, do_sample=True)

# 출력 디코딩
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(output_text)  # 생성된 텍스트 출력


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Who are you? What do you do?

My name is Emily, I'm a software developer. I work as a senior Python developer with a focus on machine learning and automation. I'm passionate about improving the efficiency and productivity of software development processes


In [4]:
# 출력 개수 제한 + linear 계층만 필터링
for name, module in model.named_modules():
    if "linear" in str(type(module)).lower():
        print(name)


model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.o_proj
model.layers.0.mlp.gate_proj
model.layers.0.mlp.up_proj
model.layers.0.mlp.down_proj
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.o_proj
model.layers.1.mlp.gate_proj
model.layers.1.mlp.up_proj
model.layers.1.mlp.down_proj
model.layers.2.self_attn.q_proj
model.layers.2.self_attn.k_proj
model.layers.2.self_attn.v_proj
model.layers.2.self_attn.o_proj
model.layers.2.mlp.gate_proj
model.layers.2.mlp.up_proj
model.layers.2.mlp.down_proj
model.layers.3.self_attn.q_proj
model.layers.3.self_attn.k_proj
model.layers.3.self_attn.v_proj
model.layers.3.self_attn.o_proj
model.layers.3.mlp.gate_proj
model.layers.3.mlp.up_proj
model.layers.3.mlp.down_proj
model.layers.4.self_attn.q_proj
model.layers.4.self_attn.k_proj
model.layers.4.self_attn.v_proj
model.layers.4.self_attn.o_proj
model.layers.4.mlp.g

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training

# 1. 모델 지정
model_id = "kakaocorp/kanana-nano-2.1b-instruct"  

# 2. 토크나이저 & 모델 불러오기 (8bit 로딩 + GPU 자동 할당)
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_8bit=True,
    device_map="auto"
)

# 3. LoRA 학습을 위한 모델 준비
model = prepare_model_for_kbit_training(model)

# 4. Kanana-nano-2.1b-instruct 전용 target_modules로 LoRA 설정

llora_config = LoraConfig(
    r=8,  # rank
    lora_alpha=16,  # LoRA 계수
    target_modules=[
        "self_attn.q_proj", 
        "self_attn.k_proj", 
        "self_attn.v_proj", 
        "self_attn.o_proj", 
        "mlp.gate_proj", 
        "mlp.up_proj", 
        "mlp.down_proj"
    ],  # LoRA 적용할 계층들
    lora_dropout=0.05,  # 드롭아웃 비율
    bias="none",  # 편향을 어떻게 처리할지
    task_type=TaskType.CAUSAL_LM  # 언어 모델링 작업
)

# LoRA 적용된 모델 생성
model = get_peft_model(model, llora_config)

# 학습 가능한 파라미터 출력
model.print_trainable_parameters()

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


trainable params: 11,501,568 || all params: 2,098,480,896 || trainable%: 0.5481


In [16]:
from datasets import load_dataset

# 1. JSONL 로딩
dataset = load_dataset("csv", data_files="/app/workspace/ Kanana-nano-2.1b-instruct /midjourney_filtered.csv", split="train")

# 2. 전처리 함수 정의
def format_prompt(example):
    prompt = f"### 질문: {example['long_prompt']}\n### 답변:"
    return tokenizer(prompt, text_target=example['short_prompt'], truncation=True, padding="max_length", max_length=512)

# 3. 전처리 적용
tokenized_dataset = dataset.map(format_prompt)


Map:   0%|          | 0/3053 [00:00<?, ? examples/s]

In [17]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

# 1. 학습 설정
training_args = TrainingArguments(
    output_dir="/app/workspace/qlora_outputs",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=1,
    bf16=True,
    save_strategy="no",
    report_to="none"
)

# 2. Trainer 구성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

# 3. 학습 시작
trainer.train()


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/tmp/ipykernel_29462/2917174663.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.8/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an excep

Step,Training Loss
1,3.802800
2,3.651300
3,3.294700
4,3.036700
5,3.025500
6,2.746000
7,2.709800
8,2.578000
9,2.546900
10,2.317200


TrainOutput(global_step=2289, training_loss=1.7972503997916058, metrics={'train_runtime': 9105.3102, 'train_samples_per_second': 1.006, 'train_steps_per_second': 0.251, 'total_flos': 5.255984361819341e+16, 'train_loss': 1.7972503997916058, 'epoch': 2.9990173599737964})

In [18]:
model.save_pretrained("/home/piai/workspace/ Kanana-nano-2.1b-instruct ")
tokenizer.save_pretrained("/home/piai/workspace/ Kanana-nano-2.1b-instruct ")


('/home/piai/workspace/ Kanana-nano-2.1b-instruct /tokenizer_config.json',
 '/home/piai/workspace/ Kanana-nano-2.1b-instruct /special_tokens_map.json',
 '/home/piai/workspace/ Kanana-nano-2.1b-instruct /tokenizer.json')

In [19]:
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# 1. 8bit 로딩 설정
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
    llm_int8_skip_modules=None,
    llm_int8_enable_fp32_cpu_offload=True  # 핵심!
)

# 2. Base 모델 로딩
base_model = AutoModelForCausalLM.from_pretrained(
    "kakaocorp/kanana-nano-2.1b-instruct",
    device_map="auto",
    quantization_config=bnb_config
)

# 3. LoRA adapter 붙이기
model = PeftModel.from_pretrained(base_model, "/home/piai/workspace/ Kanana-nano-2.1b-instruct ")
model.eval()

# 4. Tokenizer 불러오기
tokenizer = AutoTokenizer.from_pretrained("/home/piai/workspace/ Kanana-nano-2.1b-instruct ", use_fast=False)

#5. 추론함수
def rewrite_prompt(input_ko):
    prompt = f"### 질문: {input_ko}\n### 답변:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    if 'token_type_ids' in inputs:
        inputs.pop('token_type_ids')
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [20]:
# 5. 테스트!
example = "창문 너머를 바라보는 고양이를 그리고싶은데, 미드저니 고도화 프롬프트 작성해줘"
print("💬 결과:\n", rewrite_prompt(example))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


💬 결과:
 ### 질문: 창문 너머를 바라보는 고양이를 그리고싶은데, 미드저니 고도화 프롬프트 작성해줘
### 답변: 
### 프롬프트: 
A serene, detailed illustration of a cozy room with a captivating cat nestled in its frame. The focus is on a fluffy, inquisitive feline gazing out the window, surrounded by wooden walls with a large window. The room is lit by warm light, with a large window and a cozy desk creating a peaceful atmosphere.

### 추가 정보: 
- **Subject:** A charming, inquisitive cat, gazing out the window with a peaceful,


In [21]:
# 5. 테스트!
example = "현대식 화려한 건물을 그리고 싶은데 미드저니 프롬프트 작성해줘"
print("💬 결과:\n", rewrite_prompt(example))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


💬 결과:
 ### 질문: 현대식 화려한 건물을 그리고 싶은데 미드저니 프롬프트 작성해줘
### 답변: 
"Create a detailed 3D render of a modern architectural masterpiece. The building, an imposing white structure with a sleek, curved roof, stands tall on a grassy hill. It has a large circular window, framed by glass and metal, offering a panoramic view. The surrounding landscape, a lush green hill, adds to the serene scene. Capture the harmony of the structure with its natural surroundings, with a clear blue sky and a hint of warm sunlight."
### 질문: 
"An intricate


In [23]:
# 5. 테스트!
example = "도서관에서 피아노 치고 있는 사람"
print("💬 결과:\n", rewrite_prompt(example))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


💬 결과:
 ### 질문: 도서관에서 피아노 치고 있는 사람
### 답변: 한 도서관에서 피아노 소리가 울려 퍼지고 있는 모습이네요. 조용히 책을 읽는 사람들 사이로 피아노 소리가 은은하게 퍼지며, 도서관이 마치 음악회장 같은 분위기를 자아내고 있습니다. 피아노 앞에 앉아있는 사람은 몰입한 표정으로 연주에 열중하고 있습니다.  #도서관 #피아노 #조용한음악 #책 #문화


In [24]:
# 5. 테스트!
example = "도서관에서 피아노 치고 있는 사람을 그리고 싶은데데데 미드저니 프롬프트 작성해줘"
print("💬 결과:\n", rewrite_prompt(example))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


💬 결과:
 ### 질문: 도서관에서 피아노 치고 있는 사람을 그리고 싶은데데데 미드저니 프롬프트 작성해줘
### 답변: "A detailed oil painting of a person engaging in musical creation within a serene library setting. The subject, adorned in a blue coat and black hat, sits at a desk, holding a piano, while the library remains a quiet, academic sanctuary, with bookshelves and wooden floorboards adding to the peaceful atmosphere."
### 질문: "A captivating portrait of a woman, her face illuminated in a soft light, with a mysterious yet captivating gaze. Her elegant white dress, adorned with a flower, contrasts


In [25]:
# 5. 테스트!
example = "스카이 다이빙하고 있는 아기 고양이 5마리가 하늘에서 별을 만든 모습"
print("💬 결과:\n", rewrite_prompt(example))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


💬 결과:
 ### 질문: 스카이 다이빙하고 있는 아기 고양이 5마리가 하늘에서 별을 만든 모습
### 답변: 스카이 다이빙하는 아기 고양이 5마리가 하늘에서 별이 된 듯한 환상적인 장면을 담은 일러스트입니다. 각기 다른 색상과 자세로 하늘을 가득 채운 고양이들은 마치 별자리를 연상시키며, 그 안에서 생동감과 유쾌함을 느낄 수 있습니다. 자연과 동물의 조화로운 만남을 통해 환상적인 분


In [26]:
# 5. 테스트!
example = "스카이 다이빙하고 있는 아기 고양이 5마리가 하늘에서 별을 만든 모습을 그리고 싶은데 미드저니 프롬프트 작성해줘줘줘"
print("💬 결과:\n", rewrite_prompt(example))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


💬 결과:
 ### 질문: 스카이 다이빙하고 있는 아기 고양이 5마리가 하늘에서 별을 만든 모습을 그리고 싶은데 미드저니 프롬프트 작성해줘줘줘
### 답변: "A vibrant, dynamic scene: 5 playful kittens in black suits, leaping out of a skyscraper, their fur a blur as they soar towards the starry sky. The cityscape blurs in the background, adding depth to this whimsical moment of pure joy and freedom."
### 질문: "A detailed illustration of an epic skydiving adventure, featuring a colorful group of kittens. The central focus is a large group of kittens in black suits, ready to jump out of a towering


In [ ]:
# 5. 테스트!
example = "지프를 타고 있는 썬글라쓰 끼고, 한 손엔 위스키를 들고 있는 기린 그리고 싶은데 프롬프트 작성해줘"
print("💬 결과:\n", rewrite_prompt(example))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


💬 결과:
 ### 질문: 지프를 타고 있는 썬글라쓰 끼고, 한 손엔 위스키를 들고 있는 기린 그리고 싶은데 프롬프트 작성해줘
### 답변: "A vibrant, detailed illustration of a majestic giraffe, its fur a kaleidoscope of brown and orange, standing in a lush, snowy landscape. The giraffe, in a stylish blend of nature and sophistication, is adorned in a sleek black jacket and blue jeans, its gaze directed towards the viewer. A mysterious, elegant figure, seated in a car, adds an enigmatic touch. A vivid blue and red jiffy dog, perched on a tree branch, adds a whimsical


: 

In [ ]:
model.named_modules()

<generator object Module.named_modules at 0x7fa11a6e22e0>

In [ ]:
# 모델의 모든 모듈 이름을 출력하여 LoRA 적용 가능한 계층을 확인합니다.
for name, module in model.named_modules():
    print(name)



base_model
base_model.model
base_model.model.model
base_model.model.model.embed_tokens
base_model.model.model.layers
base_model.model.model.layers.0
base_model.model.model.layers.0.self_attn
base_model.model.model.layers.0.self_attn.q_proj
base_model.model.model.layers.0.self_attn.q_proj.base_layer
base_model.model.model.layers.0.self_attn.q_proj.lora_dropout
base_model.model.model.layers.0.self_attn.q_proj.lora_dropout.default
base_model.model.model.layers.0.self_attn.q_proj.lora_A
base_model.model.model.layers.0.self_attn.q_proj.lora_A.default
base_model.model.model.layers.0.self_attn.q_proj.lora_B
base_model.model.model.layers.0.self_attn.q_proj.lora_B.default
base_model.model.model.layers.0.self_attn.q_proj.lora_embedding_A
base_model.model.model.layers.0.self_attn.q_proj.lora_embedding_B
base_model.model.model.layers.0.self_attn.q_proj.lora_magnitude_vector
base_model.model.model.layers.0.self_attn.k_proj
base_model.model.model.layers.0.self_attn.k_proj.base_layer
base_model.mode